## Подготовка к работе с Википедией

Скачайте 'enwiki-20200401-pages-articles.xml.bz2' по ссылке https://meta.wikimedia.org/wiki/Data_dump_torrents — архив весит порядка 16Гб

Скачайте 'wiki.corpus' по ссылке https://yadi.sk/d/TVo-KPUbgx4vPA — это слепок памяти объекта для работы с нелемматизированной(!) Википедией

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=False)
    wiki.save('wiki.corpus')
    
for text in wiki.get_texts():
    print(text, type(text))
    break

Using preloaded object
['anarchism', 'is', 'political', 'philosophy', 'and', 'movement', 'that', 'rejects', 'all', 'involuntary', 'coercive', 'forms', 'of', 'hierarchy', 'it', 'radically', 'calls', 'for', 'the', 'abolition', 'of', 'the', 'state', 'which', 'it', 'holds', 'to', 'be', 'undesirable', 'unnecessary', 'and', 'harmful', 'the', 'timeline', 'of', 'anarchism', 'stretches', 'back', 'to', 'prehistory', 'when', 'people', 'lived', 'in', 'anarchistic', 'societies', 'long', 'before', 'the', 'establishment', 'of', 'formal', 'states', 'kingdoms', 'or', 'empires', 'with', 'the', 'rise', 'of', 'organised', 'hierarchical', 'bodies', 'skepticism', 'towards', 'authority', 'also', 'rose', 'but', 'it', 'was', 'not', 'until', 'the', 'th', 'century', 'self', 'conscious', 'political', 'movement', 'was', 'formed', 'during', 'the', 'latest', 'half', 'of', 'th', 'and', 'the', 'first', 'decades', 'of', 'th', 'century', 'the', 'anarchist', 'movement', 'flourished', 'in', 'most', 'parts', 'of', 'the', '

## Сначала рассчитаем корреляцию Спирмена с Жаккаром по окну радиуса 2

In [ ]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={i:set() for i in simlex_words}

counter=0
k=2#радиус окна

for text in wiki.get_texts():
    counter+=1
  
    for ind, word in enumerate(text):
        if word in simlex_words:
            l = ind - k if ind - k > 0 else 0
            r = ind + k + 1
            pseudodocs[word].update(text[l:r])
            
    if counter % 10000 == 0:
        simlex_pairs=[]
        jaccard_pairs=[]
        for i in f[1:]:
            ii=i.split('\t')
            w1=ii[0]
            w2=ii[1]
            try:
                jaccard_pairs.append(\
                                    len(pseudodocs[w1].intersection(pseudodocs[w2]))*1./ \
                                    len(pseudodocs[w1].union(pseudodocs[w2]))
                                    )
                simlex_pairs.append(float(ii[3]))
            except:
                print(pseudodocs[w1], pseudodocs[w2])

        coef = spearmanr(simlex_pairs, jaccard_pairs)
        print(counter, coef)
    
    if counter == 200000:
        break

После обработки первых 200000 статей можно остановиться и сделать вывод, что корреляция вряд ли станет больше 0.12

Но что интересно, и меньше 0.08 она тоже не стала, что означает, что числитель и знаменатель в коэффициенте Жаккара растут достаточно равномерно

Поэтому имеет смысл попробовать окна другого радиуса

## Затем произведём одновременный расчёт корреляции Спирмена с Жаккаром по нескольким окнам

In [5]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        if word in simlex_words:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][word].update(text[l:r])
                
    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 200000:
        break

1 10000 SpearmanrResult(correlation=0.16485923876088465, pvalue=1.6051592219945796e-07)
2 10000 SpearmanrResult(correlation=0.11076630186537834, pvalue=0.0004525067637921213)
3 10000 SpearmanrResult(correlation=0.08899130575289914, pvalue=0.004880613957089424)
4 10000 SpearmanrResult(correlation=0.07452602070841902, pvalue=0.018479444921519598)
5 10000 SpearmanrResult(correlation=0.06691854488336273, pvalue=0.03444574341758788)
1 20000 SpearmanrResult(correlation=0.16696755787891576, pvalue=1.1081451599619917e-07)
2 20000 SpearmanrResult(correlation=0.10843447177938247, pvalue=0.0005966765274165116)
3 20000 SpearmanrResult(correlation=0.08702283110789909, pvalue=0.005917711626231574)
4 20000 SpearmanrResult(correlation=0.07570368187059003, pvalue=0.016702015884979885)
5 20000 SpearmanrResult(correlation=0.06885707952098888, pvalue=0.029538693081851194)
1 30000 SpearmanrResult(correlation=0.16286050964575358, pvalue=2.2709115871863328e-07)
2 30000 SpearmanrResult(correlation=0.106107534

1 200000 SpearmanrResult(correlation=0.11080410684966215, pvalue=0.0004504625820866351)
2 200000 SpearmanrResult(correlation=0.08467571233738665, pvalue=0.007410708344663224)
3 200000 SpearmanrResult(correlation=0.07524560477862735, pvalue=0.017374742770183336)
4 200000 SpearmanrResult(correlation=0.0715996442910879, pvalue=0.023628355981486077)
5 200000 SpearmanrResult(correlation=0.07045699443926792, pvalue=0.025953196950864087)


После обработки первых 200000 статей можно сделать вывод, что наилучший результат показало окно с радиусом 1

## Наконец произведём раздельный расчёт по частям речи по нескольким окнам

In [7]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][ww][word].update(text[l:r])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 200000:
        break

1 A 10000 SpearmanrResult(correlation=0.17291892033620188, pvalue=0.06954110078372773)
1 N 10000 SpearmanrResult(correlation=0.18481679083251246, pvalue=1.57064978131956e-06)
1 V 10000 SpearmanrResult(correlation=0.09450634729802541, pvalue=0.16052547565075057)
2 A 10000 SpearmanrResult(correlation=0.13670448986759207, pvalue=0.152518135611169)
2 N 10000 SpearmanrResult(correlation=0.12362766199507362, pvalue=0.001389931291825453)
2 V 10000 SpearmanrResult(correlation=0.07415337141694049, pvalue=0.27127180418231567)
3 A 10000 SpearmanrResult(correlation=0.11982650652037191, pvalue=0.21032272798104482)
3 N 10000 SpearmanrResult(correlation=0.10164166383149141, pvalue=0.008666487247528852)
3 V 10000 SpearmanrResult(correlation=0.05892213193284733, pvalue=0.3822689610629747)
4 A 10000 SpearmanrResult(correlation=0.09806739246147103, pvalue=0.3058452580187223)
4 N 10000 SpearmanrResult(correlation=0.09188867804800506, pvalue=0.01769524336270219)
4 V 10000 SpearmanrResult(correlation=0.0427

2 V 70000 SpearmanrResult(correlation=0.07093963637294864, pvalue=0.29264530659766386)
3 A 70000 SpearmanrResult(correlation=0.1047412405122047, pvalue=0.2739382977003108)
3 N 70000 SpearmanrResult(correlation=0.10155958526248934, pvalue=0.008720872023690861)
3 V 70000 SpearmanrResult(correlation=0.050913021594544076, pvalue=0.4503724820770486)
4 A 70000 SpearmanrResult(correlation=0.09116537866541313, pvalue=0.34130111876117275)
4 N 70000 SpearmanrResult(correlation=0.09418949442441157, pvalue=0.015032812550175753)
4 V 70000 SpearmanrResult(correlation=0.03811731066512119, pvalue=0.5721184180585231)
5 A 70000 SpearmanrResult(correlation=0.08958576965932233, pvalue=0.34976659460157034)
5 N 70000 SpearmanrResult(correlation=0.09036606709727789, pvalue=0.019676092679479576)
5 V 70000 SpearmanrResult(correlation=0.0332067673914176, pvalue=0.6226399341584059)
1 A 80000 SpearmanrResult(correlation=0.17770162538242124, pvalue=0.06205692456356705)
1 N 80000 SpearmanrResult(correlation=0.16613

4 A 130000 SpearmanrResult(correlation=0.07899361371292452, pvalue=0.4098730038111129)
4 N 130000 SpearmanrResult(correlation=0.09407445037223941, pvalue=0.015157061176552574)
4 V 130000 SpearmanrResult(correlation=0.02754269648111931, pvalue=0.6831725900114339)
5 A 130000 SpearmanrResult(correlation=0.07712002191958903, pvalue=0.4211017296089018)
5 N 130000 SpearmanrResult(correlation=0.0902052410370556, pvalue=0.01989619585854566)
5 V 130000 SpearmanrResult(correlation=0.022405656181277715, pvalue=0.7398939076077697)
1 A 140000 SpearmanrResult(correlation=0.1280668001688123, pvalue=0.1804025241452368)
1 N 140000 SpearmanrResult(correlation=0.14091522464609002, pvalue=0.00026430780392394947)
1 V 140000 SpearmanrResult(correlation=0.08112706677861292, pvalue=0.22861750227785385)
2 A 140000 SpearmanrResult(correlation=0.08588685190339301, pvalue=0.37009867331104085)
2 N 140000 SpearmanrResult(correlation=0.10619110362346286, pvalue=0.006086530665786748)
2 V 140000 SpearmanrResult(correl

5 N 190000 SpearmanrResult(correlation=0.09593071377760887, pvalue=0.01325864744112122)
5 V 190000 SpearmanrResult(correlation=0.021364208596202538, pvalue=0.7515784966021003)
1 A 200000 SpearmanrResult(correlation=0.12738230293283961, pvalue=0.1827622536708083)
1 N 200000 SpearmanrResult(correlation=0.1381978660473362, pvalue=0.0003476314701701081)
1 V 200000 SpearmanrResult(correlation=0.07266715606383162, pvalue=0.28102042255295634)
2 A 200000 SpearmanrResult(correlation=0.09299509243080165, pvalue=0.3316584406759212)
2 N 200000 SpearmanrResult(correlation=0.10999123371757369, pvalue=0.004485460087239874)
2 V 200000 SpearmanrResult(correlation=0.05322460746663037, pvalue=0.4300468680720049)
3 A 200000 SpearmanrResult(correlation=0.07862503827817001, pvalue=0.4120679534223133)
3 N 200000 SpearmanrResult(correlation=0.10330242953167779, pvalue=0.00762914249506186)
3 V 200000 SpearmanrResult(correlation=0.03773506265732899, pvalue=0.5759808673252795)
4 A 200000 SpearmanrResult(correlat

После обработки первых 200000 статей можно сделать вывод, что лучший результат показали существительные

## Теперь поработаем с лемматизированной Вики

Скачайте 'wiki.lem.corpus' по ссылке https://yadi.sk/d/AsaBf1j_oFBFHw — это слепок памяти объекта для работы с лемматизированной(!) Википедией

В этом разделе также проведён раздельный расчёт по нескольким частям речи, однако далее есть раздел, где проведён расчёт, когда в псевдодокумент к существительным попадают только существительные и т.д. — такая возможность как раз является следствием лемматизации 

Т.е. на нелемматизированной Википедии в расчёте по частям речи к существительным могло попасть что угодно, а в лемматизированной можно сделать так, что попадут только существительные — в этом разделе так не сделано, но сделано далее

При лемматизации важно не забывать обрезать у слова часть речи — это не должно влиять на показатель качества, но занимает меньше памяти

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.lem.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.lem.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=True)
    wiki.save('wiki.lem.corpus')

for text in wiki.get_texts():
    print([word.decode('utf-8').split('/')[0] for word in text]) 
    break

Using preloaded object
['anarchism', 'be', 'political', 'philosophy', 'movement', 'reject', 'involuntary', 'coercive', 'form', 'hierarchy', 'radically', 'call', 'abolition', 'state', 'hold', 'be', 'undesirable', 'unnecessary', 'harmful', 'timeline', 'anarchism', 'stretch', 'back', 'prehistory', 'person', 'live', 'anarchistic', 'society', 'long', 'establishment', 'formal', 'state', 'kingdom', 'empire', 'rise', 'organise', 'hierarchical', 'body', 'skepticism', 'authority', 'also', 'rise', 'be', 'not', 'century', 'self', 'conscious', 'political', 'movement', 'be', 'form', 'latest', 'half', 'first', 'decade', 'century', 'anarchist', 'movement', 'flourish', 'most', 'part', 'world', 'have', 'significant', 'role', 'worker', 'struggle', 'emancipation', 'various', 'branch', 'anarchism', 'be', 'espouse', 'time', 'anarchist', 'take', 'part', 'several', 'revolution', 'most', 'notably', 'spanish', 'civil', 'war', 'be', 'crush', 'fascist', 'force', 'mark', 'end', 'classical', 'era', 'anarchism', 'la

Process InputQueue-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cheptil/.local/lib/python3.6/site-packages/gensim/utils.py", line 1238, in run
    self.q.put(wrapped_chunk.pop(), block=True)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 82, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt


In [3]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 200000:
        break

1 10000 SpearmanrResult(correlation=0.07415098970047042, pvalue=0.019079158785807793)
2 10000 SpearmanrResult(correlation=0.05891852908089593, pvalue=0.06267056860938328)
3 10000 SpearmanrResult(correlation=0.058309790168474374, pvalue=0.06543778293280282)
4 10000 SpearmanrResult(correlation=0.058839781606852805, pvalue=0.06302300601484578)
5 10000 SpearmanrResult(correlation=0.05866003437791815, pvalue=0.06383360294441344)
1 20000 SpearmanrResult(correlation=0.07405834888751883, pvalue=0.01922989228829983)
2 20000 SpearmanrResult(correlation=0.06740232644391354, pvalue=0.033160197075306014)
3 20000 SpearmanrResult(correlation=0.06759629674935391, pvalue=0.032656398595978825)
4 20000 SpearmanrResult(correlation=0.06847194323693834, pvalue=0.03046276127600219)
5 20000 SpearmanrResult(correlation=0.06934319045865912, pvalue=0.028406807049993554)
1 30000 SpearmanrResult(correlation=0.07629488833879643, pvalue=0.015867600342628525)
2 30000 SpearmanrResult(correlation=0.06913774895553083, p

2 200000 SpearmanrResult(correlation=0.07860306574332639, pvalue=0.012950068256234832)
3 200000 SpearmanrResult(correlation=0.07964218303227888, pvalue=0.0117990951978651)
4 200000 SpearmanrResult(correlation=0.08143668606391512, pvalue=0.010023381641166848)
5 200000 SpearmanrResult(correlation=0.08375870974816643, pvalue=0.00808021887611815)


Произошло странное — лемматизация ухудшила качество, интерпретация такая, что числитель растёт медленнее знаменателя

Что интересно — ширина окна почти не влияет на качество, но возможно это просто эффект близости к нулю

Теперь проведём такой же расчёт по частям речи

In [2]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        word=word.decode('utf-8').split('/')[0]
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][ww][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 200000:
        break

1 A 10000 SpearmanrResult(correlation=0.10157324689443369, pvalue=0.2887931522785633)
1 N 10000 SpearmanrResult(correlation=0.08833664426721247, pvalue=0.022614947454044385)
1 V 10000 SpearmanrResult(correlation=0.024389835940520473, pvalue=0.7177984678673831)
2 A 10000 SpearmanrResult(correlation=0.07744033152360189, pvalue=0.419169548181784)
2 N 10000 SpearmanrResult(correlation=0.07252988187074305, pvalue=0.06138318763609742)
2 V 10000 SpearmanrResult(correlation=0.01624581454064349, pvalue=0.8097825285522764)
3 A 10000 SpearmanrResult(correlation=0.0659004101735496, pvalue=0.49196180853781046)
3 N 10000 SpearmanrResult(correlation=0.07301632159924207, pvalue=0.05965903055623526)
3 V 10000 SpearmanrResult(correlation=0.01379328499853912, pvalue=0.8380683958853052)
4 A 10000 SpearmanrResult(correlation=0.06884023804599639, pvalue=0.4728071974192599)
4 N 10000 SpearmanrResult(correlation=0.0728858812494813, pvalue=0.060117388171303646)
4 V 10000 SpearmanrResult(correlation=0.014823764

2 V 70000 SpearmanrResult(correlation=0.01978531044063043, pvalue=0.7694006273383496)
3 A 70000 SpearmanrResult(correlation=0.09292927538888121, pvalue=0.332002258935143)
3 N 70000 SpearmanrResult(correlation=0.09011949436301793, pvalue=0.020014421162547424)
3 V 70000 SpearmanrResult(correlation=0.01830677295281438, pvalue=0.7862005462565285)
4 A 70000 SpearmanrResult(correlation=0.08869943349479359, pvalue=0.3545737176053896)
4 N 70000 SpearmanrResult(correlation=0.09177309213655996, pvalue=0.01783924999550615)
4 V 70000 SpearmanrResult(correlation=0.010991961004561233, pvalue=0.8706311168950542)
5 A 70000 SpearmanrResult(correlation=0.0926616194184047, pvalue=0.33340279212026436)
5 N 70000 SpearmanrResult(correlation=0.09250729101043878, pvalue=0.016941674275761133)
5 V 70000 SpearmanrResult(correlation=0.009959836541627331, pvalue=0.882686762609888)
1 A 80000 SpearmanrResult(correlation=0.11451287733599422, pvalue=0.23140320113499405)
1 N 80000 SpearmanrResult(correlation=0.09856823

4 A 130000 SpearmanrResult(correlation=0.07598358099576258, pvalue=0.427998453037712)
4 N 130000 SpearmanrResult(correlation=0.0974846178714233, pvalue=0.011834044163612297)
4 V 130000 SpearmanrResult(correlation=0.0007771096514226048, pvalue=0.9908139107450344)
5 A 130000 SpearmanrResult(correlation=0.07932269892252677, pvalue=0.40791903246856354)
5 N 130000 SpearmanrResult(correlation=0.09842111903273493, pvalue=0.011042445376850651)
5 V 130000 SpearmanrResult(correlation=0.001776877396336796, pvalue=0.9789977641097427)
1 A 140000 SpearmanrResult(correlation=0.09149007607222069, pvalue=0.33957715575866754)
1 N 140000 SpearmanrResult(correlation=0.10246618983060797, pvalue=0.008136713274014638)
1 V 140000 SpearmanrResult(correlation=0.0233110958668802, pvalue=0.7297833835147494)
2 A 140000 SpearmanrResult(correlation=0.08252579496265536, pvalue=0.38918797008681516)
2 N 140000 SpearmanrResult(correlation=0.09838435526363319, pvalue=0.011072612780760953)
2 V 140000 SpearmanrResult(corre

5 N 190000 SpearmanrResult(correlation=0.10711711129409801, pvalue=0.0056550263466811565)
5 V 190000 SpearmanrResult(correlation=0.00701921413447983, pvalue=0.9171734666987279)
1 A 200000 SpearmanrResult(correlation=0.09671594920070445, pvalue=0.31259082378458736)
1 N 200000 SpearmanrResult(correlation=0.11070027884714083, pvalue=0.004232827286883334)
1 V 200000 SpearmanrResult(correlation=0.020865695771119158, pvalue=0.757191862546085)
2 A 200000 SpearmanrResult(correlation=0.08283732896107882, pvalue=0.38739415017289247)
2 N 200000 SpearmanrResult(correlation=0.10742548010211052, pvalue=0.005517564965132098)
2 V 200000 SpearmanrResult(correlation=0.013878289935709555, pvalue=0.8370842958271407)
3 A 200000 SpearmanrResult(correlation=0.08570695198881045, pvalue=0.37110565605067536)
3 N 200000 SpearmanrResult(correlation=0.10690292679399975, pvalue=0.005752309115857453)
3 V 200000 SpearmanrResult(correlation=0.00742723783289791, pvalue=0.91237753823625)
4 A 200000 SpearmanrResult(corre

Существительные на окне радиуса 1 показали лучший результат

Как можно заметить, эксперименты становятся всё более вычислительно затратными, поэтому учитывая что в предыдущих расчётах показатель качества не сильно менялся, в дальнейших расчётах ограничимся 10000 документами

## Теперь посчитаем tfidf по нелемматизированной Вики и добавим в окно фильтрацию по tfidf

Скачайте 'wiki.tfidf.model' по ссылке https://yadi.sk/d/_lwvMkGZ_l9_BA

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models import TfidfModel

wiki = WikiCorpus.load("wiki.corpus")

if os.path.isfile("wiki.tfidf.model"):
    print("Using preloaded object")
    tfidf = TfidfModel.load("wiki.tfidf.model")
else:
    print("Now wait for about 9 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    tfidf = TfidfModel(wiki)
    tfidf.save('wiki.tfidf.model')

Using preloaded object


In [2]:
#Генераторы используются для синхронного доступа к документам

def next_text():
    for text in wiki.get_texts():
        yield text

n_t=next_text()

def tfidf_for_text():
    for doc in tfidf[wiki]:
        yield doc
        
t_t=tfidf_for_text()

In [ ]:
#Тест

cur_text = next(n_t)
cur_tfidf = next(t_t)
cur_dir={i:j for i,j in cur_tfidf}

for w in cur_text:
    cur_w_id=wiki.dictionary.token2id[w]
    cur_w_tfidf=cur_dir[cur_w_id]
    print(w, cur_w_tfidf)

In [4]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{t:{i:set() for i in simlex_words} for t in range(1, 11)} for k in [1, 2, 3, 4, 5]}

counter=0
#Раскоментируйте, если хотите каждый раз начинать обработку с первого документа(требуется незначительное время на инициализацию)
#n_t=next_text()
#t_t=tfidf_for_text()

while counter < 10000:
    counter+=1

    cur_text = next(n_t)
    cur_tfidf = next(t_t)
    cur_dir={i:j for i,j in cur_tfidf}
    
    for ind, word in enumerate(cur_text):
        if word in simlex_words:
            #cur_w_id=wiki.dictionary.token2id[word]
            #cur_w_tfidf=cur_dir[cur_w_id]
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                for t in range(1, 11):
                    try:
                        good_w = [item for item in cur_text[l:r] if cur_dir[wiki.dictionary.token2id[item]] > t/1000] #mitimco
                        #print(word, good_w)
                        pseudodocs[k][t][word].update(good_w)
                    except:
                        pass
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for t in range(1, 11):
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        jaccard_pairs.append(\
                                            len(pseudodocs[k][t][first_word].intersection(pseudodocs[k][t][second_word]))*1./ \
                                            len(pseudodocs[k][t][first_word].union(pseudodocs[k][t][second_word]))
                                            )
                        simlex_pairs.append(float(ii[3]))
                    except:
                        #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                        pass

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, t, counter, coef)
                

1 1 1000 SpearmanrResult(correlation=0.09523837322639907, pvalue=0.0025844162504016716)
1 2 1000 SpearmanrResult(correlation=0.09260775674986609, pvalue=0.003393080739511501)
1 3 1000 SpearmanrResult(correlation=0.09036420498077022, pvalue=0.004257674611503342)
1 4 1000 SpearmanrResult(correlation=0.0810693316092976, pvalue=0.010366227575204098)
1 5 1000 SpearmanrResult(correlation=0.08523920588554723, pvalue=0.007024371556388656)
1 6 1000 SpearmanrResult(correlation=0.08061778132206862, pvalue=0.010801901498093767)
1 7 1000 SpearmanrResult(correlation=0.07902987143799914, pvalue=0.012465831045554826)
1 8 1000 SpearmanrResult(correlation=0.07353163555748755, pvalue=0.02010679943041202)
1 9 1000 SpearmanrResult(correlation=0.07654945898986988, pvalue=0.015519728778482785)
1 10 1000 SpearmanrResult(correlation=0.07697167491535863, pvalue=0.014957530575768426)
2 1 1000 SpearmanrResult(correlation=0.12333435510103416, pvalue=9.295785120126371e-05)
2 2 1000 SpearmanrResult(correlation=0.123

5 5 2000 SpearmanrResult(correlation=0.10662387502437706, pvalue=0.0007369135967719191)
5 6 2000 SpearmanrResult(correlation=0.11082635323901237, pvalue=0.0004492637059985628)
5 7 2000 SpearmanrResult(correlation=0.11264873711273321, pvalue=0.00036055868906579214)
5 8 2000 SpearmanrResult(correlation=0.10719843128727836, pvalue=0.0006894052037117411)
5 9 2000 SpearmanrResult(correlation=0.10657231681336451, pvalue=0.0007413219996125509)
5 10 2000 SpearmanrResult(correlation=0.10953790138510046, pvalue=0.00052382706579223)
1 1 3000 SpearmanrResult(correlation=0.13836462251178985, pvalue=1.1390626418270668e-05)
1 2 3000 SpearmanrResult(correlation=0.1402800512274067, pvalue=8.575131777829324e-06)
1 3 3000 SpearmanrResult(correlation=0.13886174104670923, pvalue=1.0585246582470997e-05)
1 4 3000 SpearmanrResult(correlation=0.13228908524104188, pvalue=2.735121592796002e-05)
1 5 3000 SpearmanrResult(correlation=0.13476615644286574, pvalue=1.9223320464411188e-05)
1 6 3000 SpearmanrResult(corre

4 9 4000 SpearmanrResult(correlation=0.11135405159512163, pvalue=0.0004216826823626063)
4 10 4000 SpearmanrResult(correlation=0.11372640624493544, pvalue=0.00031609806260822867)
5 1 4000 SpearmanrResult(correlation=0.06187423258268628, pvalue=0.050573449862980985)
5 2 4000 SpearmanrResult(correlation=0.06479994474114877, pvalue=0.04058768134433761)
5 3 4000 SpearmanrResult(correlation=0.07087567782447819, pvalue=0.025079378817801787)
5 4 4000 SpearmanrResult(correlation=0.08056194739684021, pvalue=0.010856885983445934)
5 5 4000 SpearmanrResult(correlation=0.08493089120541818, pvalue=0.007233457359631252)
5 6 4000 SpearmanrResult(correlation=0.09026266094287286, pvalue=0.004301155546986344)
5 7 4000 SpearmanrResult(correlation=0.09285907181489352, pvalue=0.00330691135592268)
5 8 4000 SpearmanrResult(correlation=0.0924673964739625, pvalue=0.003442090194957268)
5 9 4000 SpearmanrResult(correlation=0.09761924366707228, pvalue=0.0020085508628783608)
5 10 4000 SpearmanrResult(correlation=0.1

4 3 6000 SpearmanrResult(correlation=0.08526095608970231, pvalue=0.0070098281905281645)
4 4 6000 SpearmanrResult(correlation=0.09300062158549059, pvalue=0.0032592584528670495)
4 5 6000 SpearmanrResult(correlation=0.09882322766213533, pvalue=0.0017645035762781753)
4 6 6000 SpearmanrResult(correlation=0.10514653402777963, pvalue=0.0008733635487622745)
4 7 6000 SpearmanrResult(correlation=0.1070998779332004, pvalue=0.0006973468924906074)
4 8 6000 SpearmanrResult(correlation=0.10957395692033636, pvalue=0.0005215925480153878)
4 9 6000 SpearmanrResult(correlation=0.11540809750736035, pvalue=0.0002568288106942497)
4 10 6000 SpearmanrResult(correlation=0.11432331119923504, pvalue=0.00029373245678489544)
5 1 6000 SpearmanrResult(correlation=0.06890287521407215, pvalue=0.029430432398691034)
5 2 6000 SpearmanrResult(correlation=0.07066347375832831, pvalue=0.025519030669001266)
5 3 6000 SpearmanrResult(correlation=0.07417710472042809, pvalue=0.019036854490899643)
5 4 6000 SpearmanrResult(correlati

3 7 8000 SpearmanrResult(correlation=0.11650829489480265, pvalue=0.0002238687119418845)
3 8 8000 SpearmanrResult(correlation=0.11807764063851768, pvalue=0.0001836579990854773)
3 9 8000 SpearmanrResult(correlation=0.12029238002426282, pvalue=0.00013831428190311677)
3 10 8000 SpearmanrResult(correlation=0.12061610770271272, pvalue=0.00013264491043825873)
4 1 8000 SpearmanrResult(correlation=0.07684851351182256, pvalue=0.015119644721329028)
4 2 8000 SpearmanrResult(correlation=0.07859161563218842, pvalue=0.012963284855208083)
4 3 8000 SpearmanrResult(correlation=0.08167151800514787, pvalue=0.009809535804407039)
4 4 8000 SpearmanrResult(correlation=0.08806695536868706, pvalue=0.0053452243024217)
4 5 8000 SpearmanrResult(correlation=0.09177222744354892, pvalue=0.0036944383788456345)
4 6 8000 SpearmanrResult(correlation=0.09817598252152737, pvalue=0.0018920924104732489)
4 7 8000 SpearmanrResult(correlation=0.10453815471523432, pvalue=0.0009360742526343537)
4 8 8000 SpearmanrResult(correlatio

3 1 10000 SpearmanrResult(correlation=0.08878066467486316, pvalue=0.004983157216261123)
3 2 10000 SpearmanrResult(correlation=0.09135246842655273, pvalue=0.0038548292822502563)
3 3 10000 SpearmanrResult(correlation=0.09364808311114045, pvalue=0.0030491539025737373)
3 4 10000 SpearmanrResult(correlation=0.09902251187778417, pvalue=0.0017268397463457087)
3 5 10000 SpearmanrResult(correlation=0.10065659313954106, pvalue=0.0014447442786863598)
3 6 10000 SpearmanrResult(correlation=0.10807832465132179, pvalue=0.0006221305886027751)
3 7 10000 SpearmanrResult(correlation=0.11043375724796448, pvalue=0.00047086412038462944)
3 8 10000 SpearmanrResult(correlation=0.11368751494126994, pvalue=0.0003176091763415682)
3 9 10000 SpearmanrResult(correlation=0.11668494920519862, pvalue=0.00021896127847211404)
3 10 10000 SpearmanrResult(correlation=0.11596676117815029, pvalue=0.00023956253869467106)
4 1 10000 SpearmanrResult(correlation=0.07461334475293678, pvalue=0.01834219229393555)
4 2 10000 SpearmanrR

Фильтрация по tfidf положительно влияет на показатель качества — наилучший результат показал порог 0.008 на окне радиуса 1, что согласуется с предыдущими расчётами


Ниже такой же расчёт делается раздельно по частям речи(три вектора: для N, A, V, но в псевдодокументах слов могут быть любые части речи)

In [3]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{t:{i:set() for i in simlex_words[w]} for t in range(1, 11)} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0
#Раскоментируйте, если хотите каждый раз начинать обработку с первого документа(требуется незначительное время на инициализацию)
n_t=next_text()
t_t=tfidf_for_text()

while counter < 10000:
    counter+=1

    cur_text = next(n_t)
    cur_tfidf = next(t_t)
    cur_dir={i:j for i,j in cur_tfidf}
    
    for ind, word in enumerate(cur_text):
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww: #if word in simlex_words:            
        #if word in simlex_words:
            #cur_w_id=wiki.dictionary.token2id[word]
            #cur_w_tfidf=cur_dir[cur_w_id]
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                for t in range(1, 11):
                    try:
                        good_w = [item for item in cur_text[l:r] if cur_dir[wiki.dictionary.token2id[item]] > t/1000] #mitimco
                        #print(word, good_w)
                        pseudodocs[k][ww][t][word].update(good_w)
                    except:
                        pass
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                for t in range(1, 11):
                    simlex_pairs=[]
                    jaccard_pairs=[]
                    for i in f[1:]:
                        ii=i.split('\t')
                        first_word=ii[0]
                        second_word=ii[1]
                        try:
                            if ii[2] == w:
                                jaccard_pairs.append(\
                                                    len(pseudodocs[k][w][t][first_word].intersection(pseudodocs[k][w][t][second_word]))*1./ \
                                                    len(pseudodocs[k][w][t][first_word].union(pseudodocs[k][w][t][second_word]))
                                                    )
                                simlex_pairs.append(float(ii[3]))
                        except:
                            #print(k, w, first_word, second_word)#print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                            pass

                    coef = spearmanr(simlex_pairs, jaccard_pairs)
                    print(k, w, t, counter, coef)
                                    
    if counter == 10000:
        break        

1 A 1 1000 SpearmanrResult(correlation=0.08982068778091036, pvalue=0.34849936566540596)
1 A 2 1000 SpearmanrResult(correlation=0.07823692673089998, pvalue=0.4143866588204048)
1 A 3 1000 SpearmanrResult(correlation=0.08323460500613115, pvalue=0.385113851666418)
1 A 4 1000 SpearmanrResult(correlation=0.0851417200698223, pvalue=0.37428041485392627)
1 A 5 1000 SpearmanrResult(correlation=0.08032666033859523, pvalue=0.4019918467840704)
1 A 6 1000 SpearmanrResult(correlation=0.07676626772342612, pvalue=0.423241634392851)
1 A 7 1000 SpearmanrResult(correlation=0.08111603068190017, pvalue=0.39736754572345645)
1 A 8 1000 SpearmanrResult(correlation=0.10004282561849766, pvalue=0.29615764826849034)
1 A 9 1000 SpearmanrResult(correlation=0.09169143404427554, pvalue=0.3385108281589758)
1 A 10 1000 SpearmanrResult(correlation=0.08856990415741461, pvalue=0.355279661081786)
1 N 1 1000 SpearmanrResult(correlation=0.10319090547163284, pvalue=0.007695155799490683)
1 N 2 1000 SpearmanrResult(correlation=0

4 N 3 1000 SpearmanrResult(correlation=0.12774917742602668, pvalue=0.0009525662267419959)
4 N 4 1000 SpearmanrResult(correlation=0.13138842029887088, pvalue=0.000676044466594081)
4 N 5 1000 SpearmanrResult(correlation=0.13000798417271472, pvalue=0.0007707421843571067)
4 N 6 1000 SpearmanrResult(correlation=0.13455082331510151, pvalue=0.0004982954685673127)
4 N 7 1000 SpearmanrResult(correlation=0.1332828465228094, pvalue=0.0005635774652343605)
4 N 8 1000 SpearmanrResult(correlation=0.1269933532665274, pvalue=0.0010217623863704997)
4 N 9 1000 SpearmanrResult(correlation=0.12367679628841828, pvalue=0.0013837748852361714)
4 N 10 1000 SpearmanrResult(correlation=0.12553809433261312, pvalue=0.0011682329272363844)
4 V 1 1000 SpearmanrResult(correlation=0.1070006795268094, pvalue=0.11187049297071848)
4 V 2 1000 SpearmanrResult(correlation=0.10270795718540084, pvalue=0.12708143544712996)
4 V 3 1000 SpearmanrResult(correlation=0.10599487889765795, pvalue=0.11529982622612471)
4 V 4 1000 Spearman

2 N 9 2000 SpearmanrResult(correlation=0.13570578787880747, pvalue=0.0004450233910603132)
2 N 10 2000 SpearmanrResult(correlation=0.13361022872598266, pvalue=0.0005460006958628723)
2 V 1 2000 SpearmanrResult(correlation=0.0534690453994507, pvalue=0.427928918549089)
2 V 2 2000 SpearmanrResult(correlation=0.05058069358050117, pvalue=0.453338407814898)
2 V 3 2000 SpearmanrResult(correlation=0.05316103459914683, pvalue=0.4305986885425568)
2 V 4 2000 SpearmanrResult(correlation=0.0356291827393586, pvalue=0.5974759474201152)
2 V 5 2000 SpearmanrResult(correlation=0.030429607206712415, pvalue=0.6520357692168396)
2 V 6 2000 SpearmanrResult(correlation=0.041992736200687386, pvalue=0.5336656090510058)
2 V 7 2000 SpearmanrResult(correlation=0.041876528732947435, pvalue=0.5347995069008383)
2 V 8 2000 SpearmanrResult(correlation=0.03176721120226186, pvalue=0.6378066995256373)
2 V 9 2000 SpearmanrResult(correlation=0.032389667563324745, pvalue=0.631229691380845)
2 V 10 2000 SpearmanrResult(correlati

5 V 8 2000 SpearmanrResult(correlation=0.03873208830749574, pvalue=0.5659321234808945)
5 V 9 2000 SpearmanrResult(correlation=0.04686020555783762, pvalue=0.48727998501502723)
5 V 10 2000 SpearmanrResult(correlation=0.04810706242164016, pvalue=0.4757555979286239)
1 A 1 3000 SpearmanrResult(correlation=0.13791582865179475, pvalue=0.1488817198760716)
1 A 2 3000 SpearmanrResult(correlation=0.13571284426055397, pvalue=0.1555443307147324)
1 A 3 3000 SpearmanrResult(correlation=0.12094399830121859, pvalue=0.20606961623233327)
1 A 4 3000 SpearmanrResult(correlation=0.10516512489737653, pvalue=0.27199047282709293)
1 A 5 3000 SpearmanrResult(correlation=0.10943666965955849, pvalue=0.25288378996536626)
1 A 6 3000 SpearmanrResult(correlation=0.08778238271070199, pvalue=0.35959050697981465)
1 A 7 3000 SpearmanrResult(correlation=0.07138798400400087, pvalue=0.4565403343831672)
1 A 8 3000 SpearmanrResult(correlation=0.08607590508446168, pvalue=0.36904226137884455)
1 A 9 3000 SpearmanrResult(correlati

4 A 1 3000 SpearmanrResult(correlation=0.11895772156702197, pvalue=0.21367228834797722)
4 A 2 3000 SpearmanrResult(correlation=0.13255552242778704, pvalue=0.16547889029660418)
4 A 3 3000 SpearmanrResult(correlation=0.13553045272259137, pvalue=0.1561057930218189)
4 A 4 3000 SpearmanrResult(correlation=0.13641678888712008, pvalue=0.15339152325138786)
4 A 5 3000 SpearmanrResult(correlation=0.1376278224584564, pvalue=0.1497403365836331)
4 A 6 3000 SpearmanrResult(correlation=0.1267329081192245, pvalue=0.18502175488026384)
4 A 7 3000 SpearmanrResult(correlation=0.1277226378427823, pvalue=0.18158618121210332)
4 A 8 3000 SpearmanrResult(correlation=0.14098010379360465, pvalue=0.13997459748635757)
4 A 9 3000 SpearmanrResult(correlation=0.1505060236608912, pvalue=0.11485884598237303)
4 A 10 3000 SpearmanrResult(correlation=0.13874232436830933, pvalue=0.14643831708401617)
4 N 1 3000 SpearmanrResult(correlation=0.10440236775375779, pvalue=0.007004699943579173)
4 N 2 3000 SpearmanrResult(correlati

...

1 N 1 8000 SpearmanrResult(correlation=0.1798992491673117, pvalue=2.981299780024431e-06)
1 N 2 8000 SpearmanrResult(correlation=0.18217162329852826, pvalue=2.2218719944561053e-06)
1 N 3 8000 SpearmanrResult(correlation=0.18561694126012418, pvalue=1.4128152623911883e-06)
1 N 4 8000 SpearmanrResult(correlation=0.18599037745240046, pvalue=1.344471999280538e-06)
1 N 5 8000 SpearmanrResult(correlation=0.1954921780250686, pvalue=3.6815951508842894e-07)
1 N 6 8000 SpearmanrResult(correlation=0.19542169950484792, pvalue=3.7180259169265236e-07)
1 N 7 8000 SpearmanrResult(correlation=0.19342522179675894, pvalue=4.906859053706226e-07)
1 N 8 8000 SpearmanrResult(correlation=0.1945021180375751, pvalue=4.2263580639863717e-07)
1 N 9 8000 SpearmanrResult(correlation=0.18724893782573102, pvalue=1.1367378711260242e-06)
1 N 10 8000 SpearmanrResult(correlation=0.18548873955496417, pvalue=1.4370370494076062e-06)
1 V 1 8000 SpearmanrResult(correlation=0.09963272355922527, pvalue=0.13892710502547223)
1 V 2 8

4 N 4 8000 SpearmanrResult(correlation=0.1033017396124649, pvalue=0.007629549310153189)
4 N 5 8000 SpearmanrResult(correlation=0.10938870582322947, pvalue=0.00471076853248102)
4 N 6 8000 SpearmanrResult(correlation=0.11708225657892353, pvalue=0.0024758510462450107)
4 N 7 8000 SpearmanrResult(correlation=0.12434519443686956, pvalue=0.0013024812677021327)
4 N 8 8000 SpearmanrResult(correlation=0.12772678553178204, pvalue=0.0009545523792973724)
4 N 9 8000 SpearmanrResult(correlation=0.13142066161434632, pvalue=0.0006739675707332116)
4 N 10 8000 SpearmanrResult(correlation=0.13515364687230003, pvalue=0.0004697918079883574)
4 V 1 8000 SpearmanrResult(correlation=0.048388100751107464, pvalue=0.4731786845165187)
4 V 2 8000 SpearmanrResult(correlation=0.05113103214721518, pvalue=0.44843274947291845)
4 V 3 8000 SpearmanrResult(correlation=0.04760386165334154, pvalue=0.4803886078271564)
4 V 4 8000 SpearmanrResult(correlation=0.0468443014083025, pvalue=0.48742794376876863)
4 V 5 8000 SpearmanrRes

2 N 7 9000 SpearmanrResult(correlation=0.15915831915539547, pvalue=3.691151935921545e-05)
2 N 8 9000 SpearmanrResult(correlation=0.1642836207393245, pvalue=2.0380543177170927e-05)
2 N 9 9000 SpearmanrResult(correlation=0.1612111397565358, pvalue=2.9160269014425497e-05)
2 N 10 9000 SpearmanrResult(correlation=0.16299242846469772, pvalue=2.3710483832838775e-05)
2 V 1 9000 SpearmanrResult(correlation=0.07893668149436318, pvalue=0.2414705281353858)
2 V 2 9000 SpearmanrResult(correlation=0.07811352607009761, pvalue=0.24642892387896032)
2 V 3 9000 SpearmanrResult(correlation=0.08060362299303231, pvalue=0.23164412550056585)
2 V 4 9000 SpearmanrResult(correlation=0.07602128635889208, pvalue=0.25934935189163844)
2 V 5 9000 SpearmanrResult(correlation=0.06897300602008946, pvalue=0.3062643168630756)
2 V 6 9000 SpearmanrResult(correlation=0.0708502440841823, pvalue=0.29325545060957786)
2 V 7 9000 SpearmanrResult(correlation=0.07114584189795561, pvalue=0.29124108867948845)
2 V 8 9000 SpearmanrResul

5 N 10 9000 SpearmanrResult(correlation=0.1246882738741647, pvalue=0.001262484283530305)
5 V 1 9000 SpearmanrResult(correlation=0.03761824942108833, pvalue=0.5771636482053306)
5 V 2 9000 SpearmanrResult(correlation=0.03672816546607147, pvalue=0.5862132069052739)
5 V 3 9000 SpearmanrResult(correlation=0.03661135222983081, pvalue=0.5874056918125703)
5 V 4 9000 SpearmanrResult(correlation=0.03834545294810764, pvalue=0.5698189799524163)
5 V 5 9000 SpearmanrResult(correlation=0.03618852121977659, pvalue=0.5917314452045449)
5 V 6 9000 SpearmanrResult(correlation=0.041148422198934, pvalue=0.5419313875412632)
5 V 7 9000 SpearmanrResult(correlation=0.043434780799343896, pvalue=0.5196959434138155)
5 V 8 9000 SpearmanrResult(correlation=0.04705434586595589, pvalue=0.4854758109148113)
5 V 9 9000 SpearmanrResult(correlation=0.050700235042143244, pvalue=0.4522702780963208)
5 V 10 9000 SpearmanrResult(correlation=0.04907252760006681, pvalue=0.46693502464734027)
1 A 1 10000 SpearmanrResult(correlation

3 V 3 10000 SpearmanrResult(correlation=0.05676355494766782, pvalue=0.3999770321336462)
3 V 4 10000 SpearmanrResult(correlation=0.057257421911801304, pvalue=0.39588292774517175)
3 V 5 10000 SpearmanrResult(correlation=0.05734902362672545, pvalue=0.3951263326475418)
3 V 6 10000 SpearmanrResult(correlation=0.06289103987028233, pvalue=0.35098050095003164)
3 V 7 10000 SpearmanrResult(correlation=0.05836768037491631, pvalue=0.3867712587354317)
3 V 8 10000 SpearmanrResult(correlation=0.06464159315704378, pvalue=0.33770790395292394)
3 V 9 10000 SpearmanrResult(correlation=0.07010549115077941, pvalue=0.298371678583122)
3 V 10 10000 SpearmanrResult(correlation=0.06565178086199822, pvalue=0.3301964344115135)
4 A 1 10000 SpearmanrResult(correlation=0.09817269972854374, pvalue=0.30532365728215166)
4 A 2 10000 SpearmanrResult(correlation=0.10226651973599576, pvalue=0.2854974769514042)
4 A 3 10000 SpearmanrResult(correlation=0.10200763937110865, pvalue=0.286725195047077)
4 A 4 10000 SpearmanrResult(

При расчёте раздельно по частям речи наилучший результат показали существительные на окне радиуса 1 при пороге 0.006 — 0.204

Подобный расчёт также необходимо сделать на лемматизированной версии Википедии(если расчёт на лемматизированной версии без фильтрации по tfidf верен, качество должно ухудшиться) — такой расчёт делается ниже

## Теперь проведём другую фильтрацию — по частям речи на лемматизированной Вики

Фильтрация заключается в том, что в окне заданного радиуса остаются только слова, относящиеся к той же части речи, что и центральное слово

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

if os.path.isfile("wiki.lem.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.lem.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=True)
    wiki.save('wiki.lem.corpus')

for text in wiki.get_texts():
    print([word.decode('utf-8').split('/') for word in text]) 
    break

Using preloaded object
[['anarchism', 'NN'], ['be', 'VB'], ['political', 'JJ'], ['philosophy', 'NN'], ['movement', 'NN'], ['reject', 'VB'], ['involuntary', 'JJ'], ['coercive', 'JJ'], ['form', 'NN'], ['hierarchy', 'NN'], ['radically', 'RB'], ['call', 'VB'], ['abolition', 'NN'], ['state', 'NN'], ['hold', 'VB'], ['be', 'VB'], ['undesirable', 'JJ'], ['unnecessary', 'JJ'], ['harmful', 'JJ'], ['timeline', 'NN'], ['anarchism', 'NN'], ['stretch', 'VB'], ['back', 'RB'], ['prehistory', 'VB'], ['person', 'NN'], ['live', 'VB'], ['anarchistic', 'JJ'], ['society', 'NN'], ['long', 'RB'], ['establishment', 'NN'], ['formal', 'JJ'], ['state', 'NN'], ['kingdom', 'NN'], ['empire', 'NN'], ['rise', 'NN'], ['organise', 'VB'], ['hierarchical', 'JJ'], ['body', 'NN'], ['skepticism', 'NN'], ['authority', 'NN'], ['also', 'RB'], ['rise', 'VB'], ['be', 'VB'], ['not', 'RB'], ['century', 'NN'], ['self', 'NN'], ['conscious', 'JJ'], ['political', 'JJ'], ['movement', 'NN'], ['be', 'VB'], ['form', 'VB'], ['latest', 'JJ']

In [8]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
simlex_pos={}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])
    if ii[2] == 'A':
        simlex_pos[ii[0]]='JJ'
        simlex_pos[ii[1]]='JJ'
    if ii[2] == 'N':
        simlex_pos[ii[0]]='NN'
        simlex_pos[ii[1]]='NN'
    if ii[2] == 'V':
        simlex_pos[ii[0]]='VB'
        simlex_pos[ii[1]]='VB'
    
pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        word_pos=word.decode('utf-8').split('/')[1]
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            if simlex_pos[word] != word_pos:
                #print(word, simlex_pos[word], word_pos)
                pass
            else:
                for k in [1, 2, 3, 4, 5]:
                    l = ind - k if ind - k > 0 else 0
                    r = ind + k + 1
                    good_w = [item.decode('utf-8').split('/')[0] for item in text[l:r] if item.decode('utf-8').split('/')[1] == word_pos]
                    #print(word, good_w)
                    pseudodocs[k][word].update(good_w)
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                    pass

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 10000:
        break        

1 1000 SpearmanrResult(correlation=0.12645279026569475, pvalue=6.126542750037106e-05)
2 1000 SpearmanrResult(correlation=0.0843109534850985, pvalue=0.007670830481299786)
3 1000 SpearmanrResult(correlation=0.07121483019740889, pvalue=0.024390305322868833)
4 1000 SpearmanrResult(correlation=0.05946490551918803, pvalue=0.06026980738303623)
5 1000 SpearmanrResult(correlation=0.050843057904007845, pvalue=0.10826920424415915)
1 2000 SpearmanrResult(correlation=0.1162131611794146, pvalue=0.00023229814651605038)
2 2000 SpearmanrResult(correlation=0.06591612832088169, pvalue=0.037245216634192835)
3 2000 SpearmanrResult(correlation=0.06928541978681978, pvalue=0.028539339754025707)
4 2000 SpearmanrResult(correlation=0.058721459466873946, pvalue=0.06355563567710754)
5 2000 SpearmanrResult(correlation=0.05924838791726884, pvalue=0.06121191290508349)
1 3000 SpearmanrResult(correlation=0.1005364008395308, pvalue=0.0014639509233100457)
2 3000 SpearmanrResult(correlation=0.07203293749977042, pvalue=0.0

Фильтрация по частям речи, возможная только при лемматизации, ухудшила качество — по какой-то причине лемматизация ухудшает качество

Что интересно — в случае лемматизации ширина окна влияет на качество, окно радиуса 1 заметно лучше всех остальных окон

Также проведём расчёт раздельно по частям речи — к каждому слову по-прежнему попадают слова той же части речи, что и оно, но теперь все слова поделены на три группы

In [2]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
simlex_pos={}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])
    if ii[2] == 'A':
        simlex_pos[ii[0]]='JJ'
        simlex_pos[ii[1]]='JJ'
    if ii[2] == 'N':
        simlex_pos[ii[0]]='NN'
        simlex_pos[ii[1]]='NN'
    if ii[2] == 'V':
        simlex_pos[ii[0]]='VB'
        simlex_pos[ii[1]]='VB'

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        word_pos=word.decode('utf-8').split('/')[1]
        word=word.decode('utf-8').split('/')[0]
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww: #if word in simlex_words:
            if simlex_pos[word] != word_pos:
                #print(word, simlex_pos[word], word_pos)
                pass
            else:
                for k in [1, 2, 3, 4, 5]:
                    l = ind - k if ind - k > 0 else 0
                    r = ind + k + 1
                    #pseudodocs[k][ww][word].update([ww.decode('utf-8').split('/')[0] for ww in text[l:r]])
                    good_w = [item.decode('utf-8').split('/')[0] for item in text[l:r] if item.decode('utf-8').split('/')[1] == word_pos]
                    #print(word, good_w)
                    pseudodocs[k][ww][word].update(good_w)                         

    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 10000:
        break        

1 A 1000 SpearmanrResult(correlation=0.11170560783171876, pvalue=0.24311896273873534)
1 N 1000 SpearmanrResult(correlation=0.13990175955288392, pvalue=0.00029291797391891547)
1 V 1000 SpearmanrResult(correlation=0.09940722187553895, pvalue=0.13982789491837253)
2 A 1000 SpearmanrResult(correlation=0.03416238312217594, pvalue=0.7218760409569154)
2 N 1000 SpearmanrResult(correlation=0.1055053459215396, pvalue=0.006424913614167532)
2 V 1000 SpearmanrResult(correlation=0.04888272829848812, pvalue=0.4686619000391409)
3 A 1000 SpearmanrResult(correlation=0.03182205511823445, pvalue=0.7402253650747052)
3 N 1000 SpearmanrResult(correlation=0.0869263553450963, pvalue=0.024874898739603368)
3 V 1000 SpearmanrResult(correlation=0.02363524387024393, pvalue=0.7261750150434121)
4 A 1000 SpearmanrResult(correlation=0.04465645644535112, pvalue=0.6416512352788646)
4 N 1000 SpearmanrResult(correlation=0.06328499284387532, pvalue=0.10272958281317486)
4 V 1000 SpearmanrResult(correlation=0.01599739391609958

3 N 7000 SpearmanrResult(correlation=0.07533717567382002, pvalue=0.05197571811112414)
3 V 7000 SpearmanrResult(correlation=0.025826693588175534, pvalue=0.7019417951567405)
4 A 7000 SpearmanrResult(correlation=0.079015552726898, pvalue=0.4097425685790831)
4 N 7000 SpearmanrResult(correlation=0.07536239086502268, pvalue=0.05189700714467669)
4 V 7000 SpearmanrResult(correlation=0.027099581400703513, pvalue=0.6880012353014746)
5 A 7000 SpearmanrResult(correlation=0.08216160733069554, pvalue=0.3912912782691961)
5 N 7000 SpearmanrResult(correlation=0.0716111420497421, pvalue=0.06475190151156653)
5 V 7000 SpearmanrResult(correlation=0.029222503723396968, pvalue=0.6649856965855516)
1 A 8000 SpearmanrResult(correlation=0.13635486129044466, pvalue=0.15358000940672392)
1 N 8000 SpearmanrResult(correlation=0.10480725110748541, pvalue=0.00678663720530146)
1 V 8000 SpearmanrResult(correlation=0.07618612803405217, pvalue=0.25831479040922933)
2 A 8000 SpearmanrResult(correlation=0.09883087014774825, p

Своё положение укрепило окно с радиусом 1, существительные(0.114) и прилагательные(0.114) показали лучшие результаты чем глаголы(0.072)

Также можно сделать вывод, что расчёта на 10000 документах вполне достаточно

## Промежуточные результаты

- Нелемматизированная - без фильтрации - все слова вместе - 1 200000 SpearmanrResult(correlation=0.11080410684966215,
- Нелемматизированная - без фильтрации - группировка по частям речи - 1 N 200000 SpearmanrResult(correlation=0.1381978660473362,


- Лемматизированная - без фильтрации - все слова вместе - 1 200000 SpearmanrResult(correlation=0.08231854917851554, pvalue=0.009241179530067959)
- Лемматизированная - без фильтрации - группировка по частям речи - 1 N 200000 SpearmanrResult(correlation=0.11070027884714083,


- Нелемматизированная - с фильтрацией по tfidf - все слова вместе - 1 8 10000 SpearmanrResult(correlation=0.1823591977724805,
- Нелемматизированная - с фильтрацией по tfidf - группировка по частям речи - 1 N 6 10000 SpearmanrResult(correlation=0.20474248263320455,


- Лемматизированная - с фильтрацией по частям речи - все слова вместе - 1 10000 SpearmanrResult(correlation=0.104834731509738,
- Лемматизированная - с фильтрацией по частям речи - группировка по частям речи - 1 N 10000 SpearmanrResult(correlation=0.11437259012735447,


Итого есть 4 параметра: лемматизация, фильтрация по tfidf, фильтрация по частям речи, группировка по частям речи(3 расчёта корреляции Спирмена)

## Фильтрация по tdidf на лемматизированной версии

Расчёт по нелемматизированной версии был связан с тем, что не сразу появился полный объект для лемматизированной версии

Скачайте tfidf для лемматизированной Википедии по ссылке — https://yadi.sk/d/TUkCRqGRiStiBg

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models import TfidfModel

wiki = WikiCorpus.load("wiki.lem.corpus")

if os.path.isfile("wiki.lem.tfidf.model"):
    print("Using preloaded object")
    tfidf = TfidfModel.load("wiki.lem.tfidf.model")
else:
    print("Now wait for about 9 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    tfidf = TfidfModel(wiki)
    tfidf.save('wiki.lem.tfidf.model')

Using preloaded object


In [2]:
#Генераторы используются для синхронного доступа к документам

def next_text():
    for text in wiki.get_texts():
        yield text

n_t=next_text()

def tfidf_for_text():
    for doc in tfidf[wiki]:
        yield doc
        
t_t=tfidf_for_text()

In [ ]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{t:{i:set() for i in simlex_words} for t in range(1, 21)} for k in [1, 2, 3, 4, 5]}

counter=0
#Раскоментируйте, если хотите каждый раз начинать обработку с первого документа(требуется незначительное время на инициализацию)
n_t=next_text()
t_t=tfidf_for_text()

while counter < 10000:
    counter+=1

    cur_text = next(n_t)
    cur_tfidf = next(t_t)
    cur_dir={i:j for i,j in cur_tfidf}
    
    for ind, word in enumerate(cur_text):
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            #cur_w_id=wiki.dictionary.token2id[word]
            #cur_w_tfidf=cur_dir[cur_w_id]
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                for t in range(1, 21):
                    try:
                        good_w = [item.decode('utf-8').split('/')[0] for item in cur_text[l:r] if \
                                  cur_dir[wiki.dictionary.token2id[item.decode('utf-8')]] > t/1000] #mitimco
                        #print(word, good_w)
                        pseudodocs[k][t][word].update(good_w)
                    except:
                        pass
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for t in range(1, 21):
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        jaccard_pairs.append(\
                                            len(pseudodocs[k][t][first_word].intersection(pseudodocs[k][t][second_word]))*1./ \
                                            len(pseudodocs[k][t][first_word].union(pseudodocs[k][t][second_word]))
                                            )
                        simlex_pairs.append(float(ii[3]))
                    except:
                        #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                        pass

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, t, counter, coef)
                

1 1 1000 SpearmanrResult(correlation=0.10008739833566885, pvalue=0.001537798195605125)
1 2 1000 SpearmanrResult(correlation=0.10324214673697055, pvalue=0.001083788990623698)
1 3 1000 SpearmanrResult(correlation=0.10210674389592808, pvalue=0.0012305913369991981)
1 4 1000 SpearmanrResult(correlation=0.09608858742474573, pvalue=0.0023634000609693783)
1 5 1000 SpearmanrResult(correlation=0.09606498478409697, pvalue=0.0023692948456897913)
1 6 1000 SpearmanrResult(correlation=0.0966133753540174, pvalue=0.0022357484941701493)
1 7 1000 SpearmanrResult(correlation=0.09371927596732678, pvalue=0.0030268205945270465)
1 8 1000 SpearmanrResult(correlation=0.09611757197208373, pvalue=0.0023561794829545044)
1 9 1000 SpearmanrResult(correlation=0.09822723763421438, pvalue=0.001881688338341651)
1 10 1000 SpearmanrResult(correlation=0.09500356420395277, pvalue=0.00264869680307841)
1 11 1000 SpearmanrResult(correlation=0.08872057931382603, pvalue=0.005012762158659761)
1 12 1000 SpearmanrResult(correlation

5 17 1000 SpearmanrResult(correlation=0.0609248252821886, pvalue=0.05422608486432315)
5 18 1000 SpearmanrResult(correlation=0.06011392368228451, pvalue=0.05751747931555011)
5 19 1000 SpearmanrResult(correlation=0.06147006612196896, pvalue=0.052102425204281134)
5 20 1000 SpearmanrResult(correlation=0.05914015379992968, pvalue=0.06168740437317491)
1 1 2000 SpearmanrResult(correlation=0.08747729041183921, pvalue=0.0056620755981316405)
1 2 2000 SpearmanrResult(correlation=0.08964858684071125, pvalue=0.004572776345878362)
1 3 2000 SpearmanrResult(correlation=0.0869242676601269, pvalue=0.005974506301864496)
1 4 2000 SpearmanrResult(correlation=0.08426433086686887, pvalue=0.00770466097213882)
1 5 2000 SpearmanrResult(correlation=0.08190131211025665, pvalue=0.009604217078112522)
1 6 2000 SpearmanrResult(correlation=0.08175695275217708, pvalue=0.009732749014490429)
1 7 2000 SpearmanrResult(correlation=0.08088728845599583, pvalue=0.010539955720923765)
1 8 2000 SpearmanrResult(correlation=0.08250

5 12 2000 SpearmanrResult(correlation=0.060073675017915856, pvalue=0.05768507496654521)
5 13 2000 SpearmanrResult(correlation=0.05866838886860916, pvalue=0.06379573751983744)
5 14 2000 SpearmanrResult(correlation=0.05910913122267072, pvalue=0.061824253547486364)
5 15 2000 SpearmanrResult(correlation=0.05873877447363659, pvalue=0.06347746063957715)
5 16 2000 SpearmanrResult(correlation=0.059974814478341756, pvalue=0.058098451964589765)
5 17 2000 SpearmanrResult(correlation=0.058299388897495114, pvalue=0.06548592547178923)
5 18 2000 SpearmanrResult(correlation=0.05805159899666991, pvalue=0.06664147561746583)
5 19 2000 SpearmanrResult(correlation=0.058903032613827275, pvalue=0.0627397947516187)
5 20 2000 SpearmanrResult(correlation=0.060180006259570006, pvalue=0.05724318794007189)


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 66, in _launch
    self.pid = os.fork(

Расчёт вылетел по памяти, но видно, что предварительные результаты не очень высокие

Фильтрация по tfidf положительно влияет на показатель качества, однако лемматизация влияет отрицательно
— в итоге общее качество ухудшилось

Расчёт раздельно по частям речи не делается, поскольку далее делается расчёт с фильтрацией по частям речи

По сравнению с аналогичным расчётом для нелемматизированной версии Википедии — как и предсказывалось(если расчёт на лемматизированной версии без фильтрации по tfidf верен, качество должно ухудшиться), качество ухудшилось: с 0.182 до 0.089, в два раза

## Фильтрация по tfidf и по частям речи на лемматизированной версии

Самый комплексный расчёт

In [1]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models import TfidfModel

wiki = WikiCorpus.load("wiki.lem.corpus")
tfidf = TfidfModel.load("wiki.lem.tfidf.model")

for text in wiki.get_texts():
    print([word.decode('utf-8').split('/') for word in text]) 
    break

[['anarchism', 'NN'], ['be', 'VB'], ['political', 'JJ'], ['philosophy', 'NN'], ['movement', 'NN'], ['reject', 'VB'], ['involuntary', 'JJ'], ['coercive', 'JJ'], ['form', 'NN'], ['hierarchy', 'NN'], ['radically', 'RB'], ['call', 'VB'], ['abolition', 'NN'], ['state', 'NN'], ['hold', 'VB'], ['be', 'VB'], ['undesirable', 'JJ'], ['unnecessary', 'JJ'], ['harmful', 'JJ'], ['timeline', 'NN'], ['anarchism', 'NN'], ['stretch', 'VB'], ['back', 'RB'], ['prehistory', 'VB'], ['person', 'NN'], ['live', 'VB'], ['anarchistic', 'JJ'], ['society', 'NN'], ['long', 'RB'], ['establishment', 'NN'], ['formal', 'JJ'], ['state', 'NN'], ['kingdom', 'NN'], ['empire', 'NN'], ['rise', 'NN'], ['organise', 'VB'], ['hierarchical', 'JJ'], ['body', 'NN'], ['skepticism', 'NN'], ['authority', 'NN'], ['also', 'RB'], ['rise', 'VB'], ['be', 'VB'], ['not', 'RB'], ['century', 'NN'], ['self', 'NN'], ['conscious', 'JJ'], ['political', 'JJ'], ['movement', 'NN'], ['be', 'VB'], ['form', 'VB'], ['latest', 'JJ'], ['half', 'NN'], ['fir

Process InputQueue-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/cheptil/.local/lib/python3.6/site-packages/gensim/utils.py", line 1238, in run
    self.q.put(wrapped_chunk.pop(), block=True)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 82, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt


In [2]:
#Генераторы используются для синхронного доступа к документам

def next_text():
    for text in wiki.get_texts():
        yield text

n_t=next_text()

def tfidf_for_text():
    for doc in tfidf[wiki]:
        yield doc
        
t_t=tfidf_for_text()

In [5]:
from scipy.stats import spearmanr

f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
simlex_pos={}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])
    if ii[2] == 'A':
        simlex_pos[ii[0]]='JJ'
        simlex_pos[ii[1]]='JJ'
    if ii[2] == 'N':
        simlex_pos[ii[0]]='NN'
        simlex_pos[ii[1]]='NN'
    if ii[2] == 'V':
        simlex_pos[ii[0]]='VB'
        simlex_pos[ii[1]]='VB' 

pseudodocs={k:{t:{i:set() for i in simlex_words} for t in range(1, 11)} for k in [1, 2, 3, 4, 5]}

counter=0
#Раскоментируйте, если хотите каждый раз начинать обработку с первого документа(требуется незначительное время на инициализацию)
n_t=next_text()
t_t=tfidf_for_text()

while counter < 10000:
    counter+=1
    
    cur_text = next(n_t)
    cur_tfidf = next(t_t)
    cur_dir={i:j for i,j in cur_tfidf}
    
    for ind, word in enumerate(cur_text):
        word_pos=word.decode('utf-8').split('/')[1]
        word=word.decode('utf-8').split('/')[0]
        if word in simlex_words:
            if simlex_pos[word] != word_pos:
                #print(word, simlex_pos[word], word_pos)
                pass
            else:
                for k in [1, 2, 3, 4, 5]:
                    l = ind - k if ind - k > 0 else 0
                    r = ind + k + 1
                    for t in range(1, 11):
                        try:
                            good_w = [item.decode('utf-8').split('/')[0] for item in cur_text[l:r] if \
                                      cur_dir[wiki.dictionary.token2id[item.decode('utf-8')]] > t/1000 and \
                                      item.decode('utf-8').split('/')[1] == word_pos] #mitimco
                            #print(word, good_w)
                            pseudodocs[k][t][word].update(good_w)
                        except:
                            pass                                                    
                
    if counter % 1000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for t in range(1, 11):
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        jaccard_pairs.append(\
                                            len(pseudodocs[k][t][first_word].intersection(pseudodocs[k][t][second_word]))*1./ \
                                            len(pseudodocs[k][t][first_word].union(pseudodocs[k][t][second_word]))
                                            )
                        simlex_pairs.append(float(ii[3]))
                    except:
                        #print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])
                        pass

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, t, counter, coef)
               

1 1000 SpearmanrResult(correlation=0.1279568925512814, pvalue=4.993125807230397e-05)
1 1000 SpearmanrResult(correlation=0.12977502109245787, pvalue=3.8874704546578936e-05)
1 1000 SpearmanrResult(correlation=0.12661473979765023, pvalue=5.9937246857499556e-05)
1 1000 SpearmanrResult(correlation=0.13093929244879923, pvalue=3.3059694250751766e-05)
1 1000 SpearmanrResult(correlation=0.12903940559784552, pvalue=4.303509447302988e-05)
1 1000 SpearmanrResult(correlation=0.12074190591831138, pvalue=0.00013050140473474178)
1 1000 SpearmanrResult(correlation=0.1215087268586183, pvalue=0.0001181241274344149)
1 1000 SpearmanrResult(correlation=0.11348152300919667, pvalue=0.0003257261746627724)
1 1000 SpearmanrResult(correlation=0.11722147123280983, pvalue=0.00020466789611367403)
1 1000 SpearmanrResult(correlation=0.11465814750266083, pvalue=0.000281843368425984)
2 1000 SpearmanrResult(correlation=0.08641287533068698, pvalue=0.006277131142151922)
2 1000 SpearmanrResult(correlation=0.0903498826765099

5 2000 SpearmanrResult(correlation=0.06846255425482221, pvalue=0.03048559466871918)
5 2000 SpearmanrResult(correlation=0.06556822062604147, pvalue=0.03826100003762837)
1 3000 SpearmanrResult(correlation=0.10121874692538373, pvalue=0.0013579537691276035)
1 3000 SpearmanrResult(correlation=0.1042892182354615, pvalue=0.0009629149898667273)
1 3000 SpearmanrResult(correlation=0.10104458692320478, pvalue=0.0013843146842945015)
1 3000 SpearmanrResult(correlation=0.10219095928993463, pvalue=0.001219102993025036)
1 3000 SpearmanrResult(correlation=0.10566358865836012, pvalue=0.0008231480655388849)
1 3000 SpearmanrResult(correlation=0.10430180469574724, pvalue=0.0009615409922677652)
1 3000 SpearmanrResult(correlation=0.10481260088524737, pvalue=0.0009072861705299255)
1 3000 SpearmanrResult(correlation=0.10252600939997127, pvalue=0.0011743694619801282)
1 3000 SpearmanrResult(correlation=0.10168882480378823, pvalue=0.0012890935095784424)
1 3000 SpearmanrResult(correlation=0.09842779351517747, pval

5 4000 SpearmanrResult(correlation=0.06917264593440023, pvalue=0.028799589033409432)
5 4000 SpearmanrResult(correlation=0.06950043875309211, pvalue=0.02804873961699784)
5 4000 SpearmanrResult(correlation=0.06879715687766048, pvalue=0.029680866174794623)
5 4000 SpearmanrResult(correlation=0.06891271142638673, pvalue=0.029407224178366047)
5 4000 SpearmanrResult(correlation=0.0671008311546372, pvalue=0.03395644991936721)
1 5000 SpearmanrResult(correlation=0.09223297148826128, pvalue=0.003525380547268126)
1 5000 SpearmanrResult(correlation=0.09381287143092489, pvalue=0.002997686241937297)
1 5000 SpearmanrResult(correlation=0.09315592204547657, pvalue=0.0032076956659454993)
1 5000 SpearmanrResult(correlation=0.09599523649748166, pvalue=0.002386793189225906)
1 5000 SpearmanrResult(correlation=0.09997710427443643, pvalue=0.0015564548694161804)
1 5000 SpearmanrResult(correlation=0.09960817691638442, pvalue=0.0016203830841334753)
1 5000 SpearmanrResult(correlation=0.1003063388150981, pvalue=0.0

5 6000 SpearmanrResult(correlation=0.06645672492439943, pvalue=0.03571235717844392)
5 6000 SpearmanrResult(correlation=0.06990514280487718, pvalue=0.027145008851782015)
5 6000 SpearmanrResult(correlation=0.07056584796328053, pvalue=0.025723521235222088)
5 6000 SpearmanrResult(correlation=0.07121359710882122, pvalue=0.0243927806743429)
5 6000 SpearmanrResult(correlation=0.07132954837178664, pvalue=0.02416096424052003)
5 6000 SpearmanrResult(correlation=0.07075319881207216, pvalue=0.025332329243257515)
5 6000 SpearmanrResult(correlation=0.07180938393540945, pvalue=0.023221811005781207)
5 6000 SpearmanrResult(correlation=0.06861024741449033, pvalue=0.030128110241069653)
1 7000 SpearmanrResult(correlation=0.10019826494041131, pvalue=0.0015192521273001065)
1 7000 SpearmanrResult(correlation=0.10254908203590364, pvalue=0.0011713454038629976)
1 7000 SpearmanrResult(correlation=0.10376356012494929, pvalue=0.0010219461695584605)
1 7000 SpearmanrResult(correlation=0.10666776049753225, pvalue=0.0

4 8000 SpearmanrResult(correlation=0.07505231286431234, pvalue=0.017665652833065634)
5 8000 SpearmanrResult(correlation=0.06963575257564772, pvalue=0.027743734999807982)
5 8000 SpearmanrResult(correlation=0.06894339712587488, pvalue=0.029334923269353788)
5 8000 SpearmanrResult(correlation=0.07017350091980065, pvalue=0.026559680952778476)
5 8000 SpearmanrResult(correlation=0.0718401344322133, pvalue=0.023162718997922258)
5 8000 SpearmanrResult(correlation=0.07383481752252034, pvalue=0.019597880570787655)
5 8000 SpearmanrResult(correlation=0.07469566849336676, pvalue=0.01821361576925449)
5 8000 SpearmanrResult(correlation=0.0749266536590702, pvalue=0.017857052031463667)
5 8000 SpearmanrResult(correlation=0.07517092133082867, pvalue=0.01748664254554952)
5 8000 SpearmanrResult(correlation=0.07597426933189429, pvalue=0.016315456120010507)
5 8000 SpearmanrResult(correlation=0.07405825269820698, pvalue=0.019230049334021174)
1 9000 SpearmanrResult(correlation=0.10517668256052962, pvalue=0.0008

4 10000 SpearmanrResult(correlation=0.07665531579428954, pvalue=0.015377059242835956)
4 10000 SpearmanrResult(correlation=0.07920806541676104, pvalue=0.012268443562606686)
4 10000 SpearmanrResult(correlation=0.07969977786815544, pvalue=0.011738036345043093)
4 10000 SpearmanrResult(correlation=0.07790555123309845, pvalue=0.013777258732507803)
5 10000 SpearmanrResult(correlation=0.0709948256324584, pvalue=0.024835406276078904)
5 10000 SpearmanrResult(correlation=0.0707633050873648, pvalue=0.02531137407803336)
5 10000 SpearmanrResult(correlation=0.07142317337381326, pvalue=0.0239751777739972)
5 10000 SpearmanrResult(correlation=0.07438979284481649, pvalue=0.01869535496711954)
5 10000 SpearmanrResult(correlation=0.0760180730641964, pvalue=0.016253622145671987)
5 10000 SpearmanrResult(correlation=0.07667786561887219, pvalue=0.015346816859942472)
5 10000 SpearmanrResult(correlation=0.07585337347731028, pvalue=0.016487186491123514)
5 10000 SpearmanrResult(correlation=0.07722908358729676, pval

Это также тяжёлый расчёт, он не был произведён полностью корректно(t должно принимать значения от 1 до 20)

Предварительный результат 0.115 не кажется положительным

## To Do

Проверить, что при лемматизации числитель растёт медленнее знаменателя по сравнению с нелемматизацией(для этого нужно настроить сериализацию результатов вычислений)

Проверить, что направления шкал совпадают — т.е. больше значит ближе(пример расчёта качества из Интернета: https://nlp.gluon.ai/examples/word_embedding_evaluation/word_embedding_evaluation.html)

Выяснить, как организовать доступ за O(1) к произвольной статье(чтобы делать расчёт на 10000 произвольных документах)

In [ ]:
import pickle

with open('pseudodocs.lem.all.pickle', 'wb') as f:
    pickle.dump(pseudodocs, f)

In [ ]:
import json

def serialize_sets(obj):
    if isinstance(obj, set):
        return list(obj)

    return obj

json_str = json.dumps(set([1,2,3]), default=serialize_sets)
print(json_str)

In [11]:
from guppy import hpy

h = hpy()

print(h.heap())

Partition of a set of 14360717 objects. Total size = 973264096 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 155122   1 464718640  48 464718640  48 dict (no owner)
     1 11076531  77 310153016  32 774871656  80 int
     2 2005028  14 64160896   7 839032552  86 numpy.float64
     3 375524   3 49296419   5 888328971  91 str
     4 327293   2 24792056   3 913121027  94 tuple
     5 165852   1 17935456   2 931056483  96 list
     6     13   0  7187848   1 938244331  96 collections.defaultdict
     7  69595   0  5456167   1 943700498  97 bytes
     8  35330   0  5113768   1 948814266  97 types.CodeType
     9  31969   0  4347784   0 953162050  98 function
<1396 more rows. Type e.g. '_.more' to view.>
